# Setting GPU

In [1]:
import os

# PROJECT_FOLDER = "/data/students_home/amoscatelli/Desktop/actionAnalysis"
# os.environ['PATH'] = "/sbin:/bin:/usr/bin:/usr/local/bin:/snap/bin:/usr/lib/jvm/java-8-oracle/bin:/usr/lib/jvm/java-8-oracle/db/bin:/usr/lib/jvm/java-8-oracle/jre/bin:/usr/lib/jvm/java-8-oracle/bin:/usr/lib/jvm/java-8-oracle/db/bin:/usr/lib/jvm/java-8-oracle/jre/bin"
# os.environ['PATH'] = PROJECT_FOLDER+"/miniconda3/bin:" + os.environ['PATH'] 
os.environ['PATH'] 

'/data/students_home/amoscatelli/.local/bin:/data/students_home/amoscatelli/Desktop/actionAnalysis/miniconda3/bin:/data/students_home/amoscatelli/Desktop/actionAnalysis/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/local/cuda/bin:/snap/bin:/usr/lib/jvm/java-8-oracle/bin:/usr/lib/jvm/java-8-oracle/db/bin:/usr/lib/jvm/java-8-oracle/jre/bin'

In [2]:
import os
# with tf.device("/GPU:0"):
# os.environ["CUDA_VISIBLE_DEVICES"]="1,2"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

## (Almost) Reproducible results

In [3]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/fchollet/keras/issues/2280#issuecomment-306959926

import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32)
# session_conf = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True

from keras import backend as K

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

/data/students_home/amoscatelli/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/students_home/amoscatelli/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/students_home/amoscatelli/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/student

In [4]:
# import tensorflow as tf
# tf.test.is_built_with_cuda()

In [5]:
# from keras import backend as K
# import tensorflow as tf
# # import os
# # with tf.device("/GPU:0"):
# # os.environ["CUDA_VISIBLE_DEVICES"]="0"
# K.tensorflow_backend._get_available_gpus()

In [6]:
# import tensorflow as tf
# from tensorflow.python.client import device_lib
# # gpus = tf.config.experimental.list_physical_devices('GPU')
# # tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
# device_lib.list_local_devices()

In [7]:
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# # config.log_device_placement = True  # to log device placement (on which device the operation ran)
#                                     # (nothing gets printed in Jupyter, only if you run it standalone)
# sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras

# Setting for training

In [8]:
DATASET_FOLDER = "/data/students_home/amoscatelli/Desktop/actionAnalysis/datasets/"

In [9]:
import keras
keras.__version__

'2.3.1'

## Load Dataset and count zero's

In [10]:
import numpy as np
import pickle
def getData(datasetName):
    with open(datasetName,'rb') as file_in:
#         features, labels, setups, cameras, performers, replications = pickle.load(file_in)
        train_set, val_set, test_set = pickle.load(file_in)
    
    #### stats ################################
    labels = ["train_set", "val_set", "test_set"]
    for i,dataset in enumerate([train_set, val_set, test_set]):
        totalsize, zero_elements =  getZeroStatsForDataset(dataset[0])
        print("{} shape: {}".format(labels[i], dataset[0].shape))
        print("{} zero elements: {}/{} ({:.2}%)".format(labels[i],zero_elements,totalsize,zero_elements*100/totalsize))
        
    return train_set, val_set, test_set

def getZeroStatsForDataset(X):
    totalsize = sum([len(x)*len(x[0])*2 for x in X])
    non_zero_elements = sum([np.count_nonzero(x) for x in X])
    zero_elements = totalsize - non_zero_elements
    return totalsize, zero_elements

## Data preprocessing functions

In [11]:
# it removes the zeros from the dataset features taking for each video the closest non-zero value 
def removeZerosFromDataset(X):
    print("removing zeros from dataset")
    for i,video in enumerate(X):
        if sum([np.count_nonzero(frame==0) for frame in video])>0:
#             print("removing zeros from video", i)
            removeZerosFromVideo(video)
    return X
    
# return the closest non zero point for the passed point
def getClosestNonZeroCoordinate(frameIdx, point, features):
    for hop in range(1,len(features)):
        previousIdx = max(0, frameIdx-hop)
        nextIdx = min(len(features)-1, frameIdx+hop)
        if all(features[previousIdx][point] != 0):
            return features[previousIdx][point]
        if all(features[nextIdx][point] != 0):
            return features[nextIdx][point]
    return [0.0,0.0] #in case that point is never found in the video
        
# it removes the zeros from the video features taking the closest non-zero values for each point
def removeZerosFromVideo(videoFeatures):
    # retrieving the index of the points which contain 0 values for each frame 
    zeroPoints = [list(set(np.where(frame == 0.)[0])) for frame in videoFeatures] 
    
    # concatenating the previous result with the frame index (discarding correct frames)
    zeroPointsCoordinates = [(i,p) for i,p in enumerate(zeroPoints) if len(p)>0]
    
    oldVideo = np.copy(videoFeatures)
    notFoundPoints = set()
    for frameIdx, pointCoordinates in zeroPointsCoordinates:
        for pointIdx in pointCoordinates:
            if pointIdx in notFoundPoints:
                videoFeatures[frameIdx][pointIdx] = [0.0,0.0]
            else:
                videoFeatures[frameIdx][pointIdx] = getClosestNonZeroCoordinate(frameIdx, pointIdx,oldVideo)
            
            if np.count_nonzero(videoFeatures[frameIdx][pointIdx]) == 0:
                notFoundPoints.add(pointIdx)

In [12]:
from sklearn.preprocessing import LabelBinarizer
def one_hot_encoding(labels):
    encoder = LabelBinarizer()
    label_strings = [str(i) for i in labels]
    oneHotLabels = encoder.fit_transform(label_strings)
    print('classes order:', encoder.classes_) 
    return oneHotLabels, encoder.classes_
    
from keras.preprocessing.sequence import pad_sequences

def paddingTrainValTest(X_train, X_val, X_test, maxLength=None):
    if maxLength is None:
         maxLength = max([len(s) for s in np.concatenate((X_train, X_val, X_test), axis=0)])
    
    # 17 if there is always only 1 person, 34 if there are videos with 2 people
    maxVideoHeigth = max([len(s[0]) for s in np.concatenate((X_train, X_val, X_test), axis=0)])
    
    for dataset in [X_train, X_val, X_test]:
        for i in range(len(dataset)):
            if dataset[i].shape[1] < maxVideoHeigth:
                videoShape = dataset[i].shape
                missingPart = (videoShape[0], maxVideoHeigth-videoShape[1], videoShape[2])
#                 dataset[i] = np.concatenate((dataset[i],np.zeros(dataset[i].shape)),axis=1)
                dataset[i] = np.concatenate((dataset[i], np.zeros(missingPart)), axis=1)
    
    
    X_train = pad_sequences(X_train, maxlen=maxLength, dtype='float32', padding='post', truncating='post', value=0.0)
    X_val = pad_sequences(X_val, maxlen=maxLength, dtype='float32', padding='post', truncating='post', value=0.0)
    X_test = pad_sequences(X_test, maxlen=maxLength, dtype='float32', padding='post', truncating='post', value=0.0)
    
    ### stats ######################################
    labels = ["train set", "val set", "test set"]
    for i, dataset in enumerate([X_train, X_val, X_test]):
        totalsize, zero_elements =  getZeroStatsForDataset(dataset)
        print("{} shape: {}".format(labels[i], dataset.shape))
        print("{} zero elements (after padding): {} ({:.2f}%)".format(labels[i],zero_elements,zero_elements*100/totalsize))
    
    return X_train, X_val, X_test

In [13]:
# import numpy as np
# #ARTIFICIAL 
# X_to_test1 = np.arange(1*17*2).reshape((1,17,2)) # 1 person
# # X_to_test1 = np.asarray([np.asarray([np.asarray([p[0]+i, p[1]+i]) for i,p in enumerate(f)]) for f in X_to_test1])

# X_to_test2 = np.arange(2*34*2).reshape((2,34,2)) # 2 people
# # X_to_test2 = np.asarray([np.asarray([np.asarray([p[0]+i+100, p[1]+i+100]) for i,p in enumerate(f)]) for f in X_to_test2])
# # X_to_test2 = X_to_test2.reshape((3,2,17,2))

# X_to_test3 = np.arange(1*17*2).reshape((1,17,2)) # 1 person
# # X_to_test3 = np.asarray([np.asarray([np.asarray([p[0]+i+200, p[1]+i+200]) for i,p in enumerate(f)]) for f in X_to_test3])

# X_to_test = np.asarray([X_to_test1, X_to_test2, X_to_test3])
# # X_to_test = np.asarray([X_to_test1,X_to_test3])
# # X_to_test = np.asarray([X_to_test2])

# # print(X_to_test)
# # print(relativeTo3GlobalBaricentersOfVideo(X_to_test))
# # X_to_test2 = X_to_test2.reshape((3,2,17,2))
# normaliseWholeDataset(X_to_test, X_to_test, X_to_test)
# # np.vstack([p.reshape((-1,2)) for p in X_to_test]).mean()


In [14]:
def normaliseWholeDataset(X_train, X_val, X_test):
    trainAndVal = np.concatenate((X_train, X_val), axis=0)
    print("normalising together train, val and test values BEFORE padding")
    trainMean = np.vstack([p.reshape((-1,2)) for p in trainAndVal]).mean() # the mean and std must be calculated only on the training data
    trainStd = np.vstack([p.reshape((-1,2)) for p in trainAndVal]).std()
    normalized_X_train = np.array([(x - trainMean)/trainStd for x in X_train])
    normalized_X_val = np.array([(x - trainMean)/trainStd for x in X_val])
    normalized_X_test = np.array([(x - trainMean)/trainStd for x in X_test])

    return normalized_X_train, normalized_X_val, normalized_X_test

In [15]:
def normalizeVideosXYInpid(dataset):
    print("normalising EACH VIDEO, considering x and y INDIPENDENTLY")

    for i in range(len(dataset)):
        xAndYVideoMean = np.mean(np.vstack(dataset[i]),axis=0)
        xAndYVideoStd = np.std(np.vstack(dataset[i]),axis=0)
        dataset[i] = (dataset[i]-xAndYVideoMean)/xAndYVideoStd

    return dataset

In [16]:
def normalizeVideos(dataset):
    print("normalising EACH VIDEO, considering x and y TOGETHER")
    for i in range(len(dataset)):
        videoMean = np.mean(np.vstack(dataset[i]))
        videoStd = np.std(np.vstack(dataset[i]))
        dataset[i] = (dataset[i]-videoMean)/videoStd

    return dataset

In [17]:
# Posenet predicts:
# 1 - nose             18
# 2 - leftEye          19
# 3 - rightEye         20
# 4 - leftEar          21
# 5 - rightEar         22

# 6 - leftShoulder     23
# 7 - rightShoulder    24
# 8 - leftElbow        25
# 9 - rightElbow       26
# 10 - leftWrist       27
# 11 - rightWrist      28

# 12 - leftHip         29
# 13 - rightHip        30
# 14 - leftKnee        31
# 15 - rightKnee       32
# 16 - leftAnkle       33
# 17 - rightAnkle      34

In [18]:
def relativeToVideoCenter(X):
    print("Adapting the data to the CENTER of each VIDEO")
    for i,frames in enumerate(X):
        # calculating the center of the whole video
        videoMean = np.mean([np.mean(features,axis=0) for features in frames], axis = 0) 
        X[i] = [frame-videoMean for frame in frames]

    return X

def relativeToPersonVideoCenter(X):
    print("Adapting the data to the video PERSON CENTER of each VIDEO")
    for i,frames in enumerate(X):
        # calculating the center of the frames points relative to the whole video
        videoMean = np.mean([np.mean(features[:17],axis=0) for features in frames], axis = 0) 

        if len(frames[0]) == 34:
            videoMean2 = np.mean([np.mean(features[17:],axis=0) for features in frames], axis = 0) 
            videoMean = np.vstack((videoMean, videoMean2)) 
            videoMean = np.repeat(videoMean, [17, 17], axis=0)
        
        X[i] = [frame-videoMean for frame in frames]

    return X

In [19]:
def relativeToFrameCenter(X):
    print("Adapting the data to the CENTER of each FRAME")
    for i_video, video in enumerate(X):
        for i_frame, frame in enumerate(video):
            frame_mean = np.mean(frame,axis=0)
            X[i_video][i_frame] = frame - frame_mean

    return X

def relativeToPersonFrameCenter(X):
    print("Adapting the data to the PERSON CENTER of each FRAME")
    for i_video, video in enumerate(X):
        for i_frame, frame in enumerate(video):
            frame_mean = np.mean(frame[:17],axis=0)
            if len(frame) == 34:
                frame_mean2 = np.mean(frame[17:],axis=0)
                frame_mean = np.vstack((frame_mean, frame_mean2)) 
                frame_mean = np.repeat(frame_mean, [17, 17], axis=0)
            X[i_video][i_frame] = frame - frame_mean

    return X

In [20]:
# [[[p[0]+i, p[1]+i]for i,p in enumerate(f)] for f in X_to_test1]

In [21]:
# PER PERSON BARICENTERS
def relativeTo5PersonalBaricentersNTURGBofVideo(X):
    print("Adapting the data to the 5 body PERSONAL BARICENTERS of each video as in NTU-RGB+D")
    for i_video,frames in enumerate(X): 
        troncoPoints = [0,1,2,3,4,5,6,11,12]
        rightArmPoints = [8,10]
        leftArmPoints = [7,9]
        rightLegPoints = [14,16]
        leftLegPoints = [13,15] 
        videoCenterFrame = np.zeros(frames[0].shape)
        for person_offset in range(0,len(frames[0]),17):
            troncoPoints = [i+person_offset for i in troncoPoints]
            rightArmPoints = [i+person_offset for i in rightArmPoints]
            leftArmPoints = [i+person_offset for i in leftArmPoints]
            rightLegPoints = [i+person_offset for i in rightLegPoints]
            leftLegPoints = [i+person_offset for i in leftLegPoints]
            troncoCenter = np.mean(np.mean(np.array(frames)[:,troncoPoints], axis=1),axis=0)
            rightArmCenter = np.mean(np.mean(np.array(frames)[:,rightArmPoints], axis=1),axis=0)
            leftArmCenter = np.mean(np.mean(np.array(frames)[:,leftArmPoints], axis=1),axis=0)
            rightLegCenter = np.mean(np.mean(np.array(frames)[:,rightLegPoints], axis=1),axis=0)
            leftLegCenter = np.mean(np.mean(np.array(frames)[:,leftLegPoints], axis=1),axis=0)

            videoCenterFrame[troncoPoints] = troncoCenter 
            videoCenterFrame[rightArmPoints] = rightArmCenter 
            videoCenterFrame[leftArmPoints] = leftArmCenter 
            videoCenterFrame[rightLegPoints] = rightLegCenter 
            videoCenterFrame[leftLegPoints] = leftLegCenter 

        X[i_video] = frames - videoCenterFrame
    return X


# GLOBAL BARICENTERS
def relativeTo5GlobalBaricentersNTURGBofVideo(X):
    print("Adapting the data to the 5 body GLOBAL BARICENTERS of each video as in NTU-RGB+D")
    for i_video,frames in enumerate(X): 
        troncoPoints = [0,1,2,3,4,5,6,11,12]
        rightArmPoints = [8,10]
        leftArmPoints = [7,9]
        rightLegPoints = [14,16]
        leftLegPoints = [13,15] 
        videoCenterFrame = np.zeros(frames[0].shape)
        if len(frames[0]) == 34:
            person_offset = 17
            troncoPoints += [i+person_offset for i in troncoPoints]
            rightArmPoints += [i+person_offset for i in rightArmPoints]
            leftArmPoints += [i+person_offset for i in leftArmPoints]
            rightLegPoints += [i+person_offset for i in rightLegPoints]
            leftLegPoints += [i+person_offset for i in leftLegPoints]
        troncoCenter = np.mean(np.mean(np.array(frames)[:,troncoPoints], axis=1),axis=0)
        rightArmCenter = np.mean(np.mean(np.array(frames)[:,rightArmPoints], axis=1),axis=0)
        leftArmCenter = np.mean(np.mean(np.array(frames)[:,leftArmPoints], axis=1),axis=0)
        rightLegCenter = np.mean(np.mean(np.array(frames)[:,rightLegPoints], axis=1),axis=0)
        leftLegCenter = np.mean(np.mean(np.array(frames)[:,leftLegPoints], axis=1),axis=0)

        videoCenterFrame[troncoPoints] = troncoCenter 
        videoCenterFrame[rightArmPoints] = rightArmCenter 
        videoCenterFrame[leftArmPoints] = leftArmCenter 
        videoCenterFrame[rightLegPoints] = rightLegCenter 
        videoCenterFrame[leftLegPoints] = leftLegCenter 

        X[i_video] = frames - videoCenterFrame
    return X

def relativeTo5GlobalBaricentersNTURGBofVideoAbs(X):
    print("ABSOLUTE VALUES of 5 GLOBAL BARICENTERS")
    return abs(relativeTo5GlobalBaricentersNTURGBofVideo(X))

def relativeTo5PersonalBaricentersNTURGBofVideoAbs(X):
    print("ABSOLUTE VALUES of 5 PERSONAL BARICENTERS")
    return abs(relativeTo5PersonalBaricentersNTURGBofVideo(X)) 

In [22]:
# def relativeTo3BaricentersOfFrame(X):
#     assert False, "Va testato"
#     print("Adapting the data to the TOP-MIDDLE-BOTTOM center of each frame")
#     for i_video, video in enumerate(X):
#         for i_frame, frame in enumerate(video):
#             X[i_video][i_frame][:5] = frame[:5] - np.mean(frame[:5], axis=0)
#             X[i_video][i_frame][5:11] = frame[5:11] - np.mean(frame[5:11], axis=0)
#             X[i_video][i_frame][11:17] = frame[11:17] - np.mean(frame[11:17], axis=0)
#             if len(frame) == 34:
#                 X[i_video][i_frame][17:22] = frame[17:22] - np.mean(frame[17:22], axis=0)
#                 X[i_video][i_frame][22:28] = frame[22:28] - np.mean(frame[22:28], axis=0)
#                 X[i_video][i_frame][28:34] = frame[28:34] - np.mean(frame[28:34], axis=0)
#     return X

In [23]:
# # model_name = "keypoint_rcnn_X_101_32x8d_FPN_3x"
# model_name = "PoseNet-101"
# datasetName = DATASET_FOLDER+model_name+"-CROSS_SUBJECT_TEST-dataset.pickle"

# train_set, val_set, test_set = getData(datasetName)

# train_set[0] = removeZerosFromDataset(train_set[0])
# val_set[0] = removeZerosFromDataset(val_set[0])
# test_set[0] = removeZerosFromDataset(test_set[0])

# labels = ["train_set", "val_set", "test_set"]
# for i,dataset in enumerate([train_set, val_set, test_set]):
#     totalsize, zero_elements =  getZeroStatsForDataset(dataset[0])
#     print("{} shape: {}".format(labels[i], dataset[0].shape))
#     print("{} zero elements: {}/{} ({:.2}%)".format(labels[i],zero_elements,totalsize,zero_elements*100/totalsize))

    
# X_to_test = train_set[0][0]
# X_to_test = X_to_test[:2]
# X_to_test = np.expand_dims(X_to_test, axis=0)
# print("X_to_test shape", X_to_test.shape)

In [24]:
def relativeTo3PersonalBaricentersOfVideo(X):
    print("Adapting the data wrt to the 3 PERSONAL BARICENTERS of each VIDEO")
    
    for i_video,frames in enumerate(X):        
        videoUpperMeanP1 = np.mean([np.mean(features[:5], axis=0) for features in frames], axis = 0) 
        videoMiddleMeanP1 = np.mean([np.mean(features[5:11], axis=0) for features in frames], axis = 0) 
        videoBottomMeanP1 = np.mean([np.mean(features[11:17], axis=0) for features in frames], axis = 0) 
        if len(frames[0]) == 34:
            videoUpperMeanP2 = np.mean([np.mean(features[17:22], axis=0) for features in frames], axis = 0) 
            videoMiddleMeanP2 = np.mean([np.mean(features[22:28], axis=0) for features in frames], axis = 0) 
            videoBottomMeanP2 = np.mean([np.mean(features[28:34], axis=0) for features in frames], axis = 0) 
        
        for i_frame, frame in enumerate(frames):
            X[i_video][i_frame][:5] = frame[:5] - videoUpperMeanP1
            X[i_video][i_frame][5:11] = frame[5:11] - videoMiddleMeanP1
            X[i_video][i_frame][11:17] = frame[11:17] - videoBottomMeanP1
            if len(frame) == 34:
                X[i_video][i_frame][17:22] = frame[17:22] - videoUpperMeanP2
                X[i_video][i_frame][22:28] = frame[22:28] - videoMiddleMeanP2
                X[i_video][i_frame][28:34] = frame[28:34] - videoBottomMeanP2
                
    return X

def relativeTo3GlobalBaricentersOfVideo(X):
    print("Adapting the data wrt to the 3 GLOBAL BARICENTERS of each VIDEO")
    
    for i_video,frames in enumerate(X):        
        upperMean = np.mean([np.mean(features[:5], axis=0) for features in frames], axis = 0) 
        middleMean = np.mean([np.mean(features[5:11], axis=0) for features in frames], axis = 0) 
        bottomMean = np.mean([np.mean(features[11:17], axis=0) for features in frames], axis = 0) 
        if len(frames[0]) == 34:
            upperMeanP2 = np.mean([np.mean(features[17:22], axis=0) for features in frames], axis = 0) 
            middleMeanP2 = np.mean([np.mean(features[22:28], axis=0) for features in frames], axis = 0) 
            bottomMeanP2 = np.mean([np.mean(features[28:34], axis=0) for features in frames], axis = 0) 
            upperMean = (upperMean + upperMeanP2)/2.
            middleMean = (middleMean+middleMeanP2)/2.
            bottomMean = (bottomMean+bottomMeanP2)/2.
        
        for i_frame, frame in enumerate(frames):
            X[i_video][i_frame][:5] = frame[:5] - upperMean
            X[i_video][i_frame][5:11] = frame[5:11] - middleMean
            X[i_video][i_frame][11:17] = frame[11:17] - bottomMean
            if len(frame) == 34:
                X[i_video][i_frame][17:22] = frame[17:22] - upperMean
                X[i_video][i_frame][22:28] = frame[22:28] - middleMean
                X[i_video][i_frame][28:34] = frame[28:34] - bottomMean
                
    return X

def relativeTo3PersonalBaricentersOfVideoAbs(X):
    print("Adapting the data as the ABSOLUTE values wrt the 3 PERSONAL BARICENTERS of each VIDEO ")
    return abs(relativeTo3PersonalBaricentersOfVideo(X))

def relativeTo3GlobalBaricentersOfVideoAbs(X):
    print("Adapting the data as the ABSOLUTE values wrt the 3 GLOBAL BARICENTERS of each VIDEO ")
    return abs(relativeTo3GlobalBaricentersOfVideo(X))

In [25]:
def relativeTo17BaricentersOfVideo(X):
    print("Adapting the data as the difference to the center of EACH KEYPOINT in the VIDEO")   
    for i_video,frames in enumerate(X): 
        video_mean_by_points = np.mean(X[i_video],axis=0)
        X[i_video] = X[i_video] - video_mean_by_points
    return X

def relativeTo17BaricentersOfVideoAbs(X):
    return abs(relativeTo17BaricentersOfVideo(X))

In [26]:
# def relativeToNextFrameOLD(X):
#     print("Adapting the data to the next frame")
#     newX = []
#     for frames in X:
#         motions = []
#         for i in range(len(frames)-1):
#             motions.append(np.array(frames[i+1])-np.array(frames[i]))
#         newX.append(motions)
#     return np.array(newX)

In [27]:
# builder function which return a function to calculate the difference each N frames
def relativeToNextFrameBuilder(step):
    step = step
    
    # it smooths the dataset following the Savitzky-Golay algorithm
    def relativeToNextFrame(X):
        print("Adapting the data to the next",step,"frame")
        newX = []
        for frames in X:
            # repeat the last frame of the video as much as the number of steps
            frames = np.concatenate((frames,np.repeat(np.expand_dims(frames[-1],axis=0),step,axis=0)))
            motions = np.array([frames[i+step]-frames[i] for i in range(len(frames)-step-1)])
            newX.append(motions)
        return np.array(newX)
    
    if step>1:
        relativeToNextFrame.__name__ = "relativeToNextFrame_"+str(step)
    return relativeToNextFrame
  

In [28]:
def cumulativeDifferences(X):
        print("Adapting the data as the CUMULATIVE DIFFERENCES of consecutive frames")
        newX = []
        for frames in X:
            motions = frames - frames[0]
            newX.append(motions)
        return np.array(newX)

In [29]:
import math
def euclDistance(point1,point2):
    return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

def relativeToPointDistances(X):
    print("Adapting the data to the DISTANCES between points of each person in each FRAME")
    newX = []
    for videoIdx,video in enumerate(X):
        if videoIdx % 100 == 0:
            print("{}/{} video done".format(videoIdx,len(X)))
        
        videoDistances = np.zeros((len(video),len(video[0]),17))
        
        for fIdx, frame in enumerate(video):
            distances = [[euclDistance(frame[i],frame[j]) for j in range(17)] for i in range(17)]
            if len(frame) == 34:
                distancesP2 = [[euclDistance(frame[i],frame[j]) for j in range(17,34)] for i in range(17,34)]
                distances = np.concatenate((distances,distancesP2),axis = 0)
            videoDistances[fIdx] = distances
        
        newX.append(videoDistances)
    return np.array(newX)

In [30]:
def preprocessData(train_set, val_set, test_set, preprocess_functions = None, normaliseDatasetGlobally = False):
    X_train, X_val, X_test = train_set[0], val_set[0], test_set[0]
    # one hot encoding
    y_train, encoding_train = one_hot_encoding(train_set[1])
    y_val, encoding_val = one_hot_encoding(val_set[1])
    y_test, encoding_test = one_hot_encoding(test_set[1])
    
    assert all(encoding_train == encoding_val)
    assert all(encoding_val == encoding_test)

    # preprocess
    if preprocess_functions is not None:
        for preprocess_function in preprocess_functions:
            X_train = preprocess_function(X_train)
            X_val = preprocess_function(X_val)
            X_test = preprocess_function(X_test)
            
    # global normalization of dataset
    # avoid this normalization if already normalized the videos individually
    if normaliseDatasetGlobally:
        X_train, X_val, X_test = normaliseWholeDataset(X_train,X_val,X_test)

    #padding
    X_train, X_val, X_test = paddingTrainValTest(X_train, X_val, X_test)

    
    ## reshaping after padding ###
    X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2] * X_train.shape[3])
    X_val = X_val.reshape(X_val.shape[0],X_val.shape[1],X_val.shape[2] * X_val.shape[3])
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2] * X_test.shape[3])
    
#     ## reshaping without padding ###
#     for dataset in [X_train, X_val, X_test]:
#         for i in range(len(dataset)):
#             dataset[i] = dataset[i].reshape(dataset[i].shape[0], 
#                                             dataset[i].shape[1] * dataset[i].shape[2])

    encoding_train = [int(i) for i in encoding_train]

    return X_train, y_train, X_val, y_val, X_test, y_test, encoding_train

In [31]:
import scipy
from scipy import signal

# builder function which returs a Savitzky-Golay smoother function with the passed arguments
def smoothingPoints(window, order):
    movingWindow = window
    polynomialOrder = order
    
    # it smooths the dataset following the Savitzky-Golay algorithm
    def smooth(X):
        for vidIdx, video in enumerate(X):
            if vidIdx % 500 == 0:
                print("smooting video {}/{}".format(vidIdx,len(X)))
            for p in range(len(X[0][0])): # X[0][0] == 17 or 34 -> the number of points
                x, y = zip(*[(f[p][0], f[p][1]) for f in video])
                
                #     Savitzky-Golay
                smooth_x = signal.savgol_filter(x, movingWindow, polynomialOrder)
                smooth_y = signal.savgol_filter(y, movingWindow, polynomialOrder)
                
                # placing the smoothed series
                for i,frame in enumerate(video):
                    frame[p] = [smooth_x[i], smooth_y[i]]
        return X
    
    smooth.__name__ = "smooth_"+str(window)+"_"+str(order)
        
    return smooth

In [32]:
import random
import numpy as np
from random import gauss

def augmentData(dataset, xAxis = None, mirroring = False, std_jittering = 0):
    X = dataset[0]
    y = dataset[1]
    print("## OLD LENGHT OF DATSET:",len(X))
    
    # check that the X and labels have the same length
    assert len(X) == len(y)
    
    # MIRRORING
    if mirroring:
        print("## Applying MIRRORING to dataset...")
        if xAxis == 0:
            mirrX = np.asarray([ np.asarray([ np.asarray([ np.asarray([-p[0],p[1]],dtype=np.float32) for p in f]) for f in v]) for v in X])
        elif xAxis == 1:
            mirrX = np.asarray([ np.asarray([ np.asarray([ np.asarray([p[0],-p[1]],dtype=np.float32) for p in f]) for f in v]) for v in X])
        else:
            raise Exception("xAxis can be only 0 or 1")

        X = np.concatenate((X,mirrX))
        y = np.concatenate((y,y))
    
    
    # JITTERING
    if std_jittering > 0:
        print("## Applying JITTERING to dataset...")
        jitterX = np.asarray([np.asarray([ np.asarray([ 
                                np.asarray([p[0]+gauss(0,std_jittering), p[1]+gauss(0,std_jittering)], dtype=np.float32) for p in f]) 
                              for f in v]) for v in X])
        
        X = np.concatenate((X,jitterX))
        y = np.concatenate((y,y))
    
    print("## NEW LENGHT OF DATSET:",len(X))
    return [X,y]

In [33]:
# import random
# a =[1,2,3,4,5,6,7,8,9]
# b =[11,12,13,14,15,16,17,18,19]
# data = list(zip(a,b))
# random.shuffle(data)
# [i[0] for i in data[:5]]

# Setting data

In [39]:
SAVED_MODEL_FOLDER = "/data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/"
DATASET_FOLDER = "/data/students_home/amoscatelli/Desktop/actionAnalysis/datasets/"

# DATASET_FOLDER = "/content/gdrive/My Drive/actionAnalysis/datasets/"
# SAVED_MODEL_FOLDER = "/content/gdrive/My Drive/actionAnalysis/savedModels/"
      
folder_where_save1 = "Cross_subject/"
folder_where_save2 = "Cross_view/"        

# folder_where_save1 = "Cross_view_lrScan/"        
# folder_where_save1 = "Cross_subject_lrScan/"     
# folder_where_save2 = "Cross_subject_lrScan/"
# SCAN_EPOCHS = 10

# folder_where_save1 = "Cross_view_tough/" 
# folder_where_save2 = "Cross_view_tough/" 

# folder_where_save1 = "Cross_subject_tough/"
# folder_where_save2 = "Cross_view_tough/"

# folder_where_save1 = "Cross_view_test/" 
# folder_where_save2 = "Cross_subject_test/"

# folder_where_save1 = "Cross_view_mini/"     
# folder_where_save2 = "Cross_subject_mini/"  

# folder_where_save1 = "Senesi/"
# folder_where_save2 = "Cross_subject/"

# folder_where_save1 = "Cross_view/"
# folder_where_save2 = "top-models/"


MODEL_NAME_VALUES = [
    "PoseNet-101"
#     ,"keypoint_rcnn_X_101_32x8d_FPN_3x"
    ]

MERGING_TRAIN_VAL = True


CONTINUE_TRAINING = None
# CONTINUE_TRAINING = "keypoint_rcnn_X_101_32x8d_FPN_3x-3L-HU_64-LR_0.01-OFF_0.001-POW_1.5-removeZerosFromDataset-relativeTo3BaricentersOfVideo-normalizeVideos-drop-0.1-rec_drop-0.1-THEN_drop_0.05"

PATIENCE = 100
HIDDEN_UNITS_VALUES = [64]

#                         [0.04,0.039,0.038,0.037,0.036,0.035,0.034,0.033,0.032,0.031,
#                         0.03,0.029,0.028,0.027,0.026,0.025,0.024,0.023,0.022,0.021,
#                         0.02,0.019,0.018,0.017,0.016,0.015,0.014,0.013,0.012,0.011,
#                         0.01,0.009,0.008,0.007,0.006,0.005,0.004,0.003,0.002,0.001]
BATCH_SIZE = 600
REFERENCE_EPOCHS = 250 #the earlystop will eventually stop the training
EPOCHS = 2000

MIRRORING_VALUES = [False,True]
STD_JITTERING_VALUES = [0]


USE_SCHEDULER = True
LEARNING_RATE_VALUES = [0.01]

###### POLYNOMIAL SCHEDULER #############
LR_OFFSET = 0.001
LR_POWER = 2
def polynomialScheduler(epoch, lr):
    if epoch < REFERENCE_EPOCHS:
        decay = (1 - (epoch / float(REFERENCE_EPOCHS)))  ** LR_POWER
        alpha = LEARNING_RATE * decay
        return float(alpha)+LR_OFFSET
    else:
        return LR_OFFSET
scheduler = polynomialScheduler
##########################################

USE_LSTM = False
USE_CuDNNLSTM = not USE_LSTM

REGULARIZER_VALUES = [0.00001,
                      0.000001]  

DROPOUT_VALUES = [0.15,0.1,0.2] # [] == don't load

LOAD_FROM_PREVIOUS_DROPOUT = [] # [] == don't load

LSTM_LAYERS_VALUES = [3]
PREPROCESS_FUNCTION_TO_TEST = [
    
     (folder_where_save1,[removeZerosFromDataset,normalizeVideosXYInpid,relativeToNextFrameBuilder(1)],True)
    ,(folder_where_save1,[removeZerosFromDataset,normalizeVideosXYInpid,relativeToNextFrameBuilder(3)],True)
#     ,(folder_where_save1,[removeZerosFromDataset,normalizeVideosXYInpid,relativeToNextFrameBuilder(5)],True)
#      ,(folder_where_save2,[removeZerosFromDataset,relativeToNextFrameBuilder(1)],True)
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeTo3GlobalBaricentersOfVideo],True)
     ,(folder_where_save1,[removeZerosFromDataset,normalizeVideosXYInpid,relativeTo3GlobalBaricentersOfVideo],True)
    
     ,(folder_where_save2,[removeZerosFromDataset,normalizeVideosXYInpid,relativeToNextFrameBuilder(1)],True)
    ,(folder_where_save2,[removeZerosFromDataset,normalizeVideosXYInpid,relativeToNextFrameBuilder(3)],True)
     ,(folder_where_save2,[removeZerosFromDataset,normalizeVideosXYInpid,relativeTo3GlobalBaricentersOfVideo],True)
    
    
#      ,(folder_where_save2,[removeZerosFromDataset,relativeTo3GlobalBaricentersOfVideo],True)
     
#      (folder_where_save1,[removeZerosFromDataset,relativeTo3PersonalBaricentersOfVideo,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeTo3PersonalBaricentersOfVideo,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeTo5GlobalBaricentersNTURGBofVideo,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeTo5GlobalBaricentersNTURGBofVideo,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeTo5PersonalBaricentersNTURGBofVideo,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeTo5PersonalBaricentersNTURGBofVideo,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeToFrameCenter,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeToFrameCenter,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeToVideoCenter,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeToVideoCenter,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeToPersonVideoCenter,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeToPersonVideoCenter,normalizeVideos])
    

    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeToNextFrameBuilder(3),normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeToNextFrameBuilder(3),normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeToNextFrameBuilder(7),normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeToNextFrameBuilder(7),normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeToNextFrameBuilder(15),normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeToNextFrameBuilder(15),normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeToPersonFrameCenter,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeToPersonFrameCenter,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,cumulativeDifferences,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,cumulativeDifferences,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeTo5PersonalBaricentersNTURGBofVideoAbs,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeTo5PersonalBaricentersNTURGBofVideoAbs,normalizeVideos])
    
    
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeTo3PersonalBaricentersOfVideoAbs,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeTo3PersonalBaricentersOfVideoAbs,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeTo17BaricentersOfVideo,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeTo17BaricentersOfVideo,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeTo17BaricentersOfVideoAbs,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeTo17BaricentersOfVideoAbs,normalizeVideos])
    
#      ,(folder_where_save1,[removeZerosFromDataset,relativeToPointDistances,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,relativeToPointDistances,normalizeVideos])
    
    
    
#      ,(folder_where_save1,[])
#      ,(folder_where_save2,[])
#      ,(folder_where_save1,[normalizeVideos])
#      ,(folder_where_save2,[normalizeVideos])
#      ,(folder_where_save1,[normalizeVideosXYInpid])
#      ,(folder_where_save2,[normalizeVideosXYInpid])
#      ,(folder_where_save1,[removeZerosFromDataset,normalizeVideos])
#      ,(folder_where_save2,[removeZerosFromDataset,normalizeVideos])
#      ,(folder_where_save1,[removeZerosFromDataset,normalizeVideosXYInpid])
#      ,(folder_where_save2,[removeZerosFromDataset,normalizeVideosXYInpid])
    
     
    # TECNIQUES

# relativeToVideoCenter
# relativeToPersonVideoCenter
# relativeToFrameCenter
# relativeToPersonFrameCenter
# relativeTo5BaricentersNTURGBofVideo
# relativeTo5BaricentersNTURGBofVideoAbs
# relativeTo3BaricentersOfVideo
# relativeTo3BaricentersOfVideoAbs
# relativeTo17BaricentersOfVideo
# relativeTo17BaricentersOfVideoAbs
# relativeToNextFrameBuilder(1)
# relativeToNextFrameBuilder(3)
# relativeToNextFrameBuilder(7)
# relativeToNextFrameBuilder(15)
# cumulativeDifferences
# relativeToPointDistances
]

if USE_LSTM:
    REGULARIZER_VALUES = [0]
if not USE_SCHEDULER:
    LR_OFFSET = 0
    LR_POWER = 0
 
    

print(len(MIRRORING_VALUES),"MIRRORING_VALUES:", MIRRORING_VALUES)
print(len(STD_JITTERING_VALUES),"STD_JITTERING_VALUES:", STD_JITTERING_VALUES)
print("NORMAL LSTM" if USE_LSTM else "CuDNNLSTM")
print(len(MODEL_NAME_VALUES),"MODEL_NAME_VALUES:", MODEL_NAME_VALUES)
print(len(LSTM_LAYERS_VALUES),"LSTM_LAYERS_VALUES LAYERS:", LSTM_LAYERS_VALUES)
print(len(HIDDEN_UNITS_VALUES), "HIDDEN UNITS",HIDDEN_UNITS_VALUES)
print(len(REGULARIZER_VALUES),"REGULARIZER_VALUES",REGULARIZER_VALUES)
print(len(DROPOUT_VALUES),"DROPOUT_VALUES",DROPOUT_VALUES)
print(len(PREPROCESS_FUNCTION_TO_TEST),"PREPROCESS_FUNCTION_TO_TEST")


numberOfTests = len(MIRRORING_VALUES) * len(STD_JITTERING_VALUES) * len(MODEL_NAME_VALUES)* len(LEARNING_RATE_VALUES)*len(HIDDEN_UNITS_VALUES) * len(PREPROCESS_FUNCTION_TO_TEST)*len(DROPOUT_VALUES)*len(REGULARIZER_VALUES)*len(LSTM_LAYERS_VALUES)
print()
if MERGING_TRAIN_VAL:
    print("!! MERGING TRAIN+VAL !!\n")
    
    
print("EPOCHS:", EPOCHS)
print("REFERENCE_EPOCHS:", REFERENCE_EPOCHS)
print("PATIENCE:", PATIENCE)
print()
    
print(len(LEARNING_RATE_VALUES),"LEARNING_RATE_VALUES",LEARNING_RATE_VALUES)
if USE_SCHEDULER:
#     print("\nUSE_SCHEDULER\n\tDECAY_RATE:", DECAY_RATE, "\n\tDECAY_STEP:", DECAY_STEP)
    print(scheduler.__name__)
    print("LR_OFFSET:",LR_OFFSET)
    print("LR_POWER:",LR_POWER)
print("BATCH_SIZE:", BATCH_SIZE)
print("CONTINUE_TRAINING:",CONTINUE_TRAINING)
print("\nNumber of tests:", numberOfTests)
SAVED_MODEL_FOLDER, folder_where_save1, folder_where_save2


2 MIRRORING_VALUES: [False, True]
1 STD_JITTERING_VALUES: [0]
CuDNNLSTM
1 MODEL_NAME_VALUES: ['PoseNet-101']
1 LSTM_LAYERS_VALUES LAYERS: [3]
1 HIDDEN UNITS [64]
2 REGULARIZER_VALUES [1e-05, 1e-06]
3 DROPOUT_VALUES [0.15, 0.1, 0.2]
6 PREPROCESS_FUNCTION_TO_TEST

!! MERGING TRAIN+VAL !!

EPOCHS: 2000
REFERENCE_EPOCHS: 250
PATIENCE: 100

1 LEARNING_RATE_VALUES [0.01]
polynomialScheduler
LR_OFFSET: 0.001
LR_POWER: 2
BATCH_SIZE: 600
CONTINUE_TRAINING: None

Number of tests: 72


('/data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/',
 'Cross_subject/',
 'Cross_view/')

## sequential trainings

In [40]:
def cleanForExcel(text):
    replacements = (
        ("relativeToVideoCenter","VIDEO"),
        ("relativeToPersonVideoCenter","VIDEO_PERS"),        
        ("relativeToFrameCenter","FRAME"),        
        ("relativeToPersonFrameCenter","FRAME_PERS"),  
        ("relativeTo3BaricentersOfVideo","3BAR_PERS"), 
        ("relativeTo3GlobalBaricentersOfVideo","3BAR_GLOB"),
        ("relativeTo3PersonalBaricentersOfVideo","3BAR_PERS"),
        
        ("relativeToUpperMiddleBottomCentersOfVideo","3BAR"),        
        ("relativeTo5BaricentersNTURGBofVideo","5BAR_OLD"),      
        ("relativeTo5PersonalBaricentersNTURGBofVideo","5BAR_PERS"),
        ("relativeTo5GlobalBaricentersNTURGBofVideo","5BAR_GLOB"),
        
        ("relativeTo17BaricentersOfVideo","17BAR"),
        ("relativeToPointDistances","DIST_REL"),
        ("relativeToNextFrame","NEXT"),
        ("cumulativeDifferences","DIST_CUM"),
        ("removeZerosFromDataset","rimoz_0"), 
        ("normalizeVideosXYInpid","normXY"),
        ("normalizeVideos","norm"),     
        ("Abs","_ASS"),    
        ("CuDNNLSTM","cuda"),
        (".pickle",""),
        (".h5",""),
        (".",","),
        ("PoseNet-101","pos"),
        ("keypoint_rcnn_X_101_32x8d_FPN_3x","det"),
        ("-drop-0-","-"),
        ("-rec_drop-0-","-"),
        ("-reg-0#",""),
        ("-reg-0-","-"),
        ("#","")
    )
    text = text+"#"
    for r in replacements:
        text = text.replace(*r)
    return text


In [41]:
def getSaveFileName(modelName,preprocess_functions,learning_rate, offset, power, numberOfLSTMLayers, useCudaLSTM, hiddenUnits, regularizerValue, dropOut, recurrentDropOut, std_jittering, mirroring, normalizeGlobally = False):
                                                                   
    saveFileName = modelName+"-{}L".format(numberOfLSTMLayers+1)
    saveFileName += "-CuDNNLSTM" if useCudaLSTM else ""
    saveFileName += "-HU_"+str(hiddenUnits)
    saveFileName += "-LR_"+str(learning_rate)
    saveFileName += "-OFF_"+str(offset)
    saveFileName += "-POW_"+str(power)
    saveFileName += "-MIRR" if mirroring else ""
    saveFileName += "-STD_JIT_"+str(std_jittering) if std_jittering>0 else "" 
    
    if preprocess_functions is not None:
        for function in preprocess_functions:
            saveFileName += "-{}".format(function.__name__) 
                             
    if normalizeGlobally:
        saveFileName += "-glob_norm"
        
    saveFileName += "-drop-"+str(dropOut)
    
    if useCudaLSTM:
        saveFileName += "-reg-"+str(regularizerValue)
    else:
        saveFileName += "-rec_drop-"+str(recurrentDropOut)
        
    
    
    return saveFileName

In [42]:
from sklearn.metrics import accuracy_score
def getValTestAccuracy(model,x_val,y_val,x_test,y_test, label_order = None):
    if label_order is None:
        raise Exception("define label order for val test accuracy evaluation")
    y_val_pred = [label_order[i] for i in model.predict_classes(x_val)]
    y_val_true = [label_order[np.argmax(i)] for i in y_val]
    y_test_pred = [label_order[i] for i in model.predict_classes(x_test)]
    y_test_true = [label_order[np.argmax(i)] for i in y_test]
    val_acc = accuracy_score(y_pred=y_val_pred,y_true=y_val_true)
    test_acc = accuracy_score(y_pred=y_test_pred,y_true=y_test_true)
    return val_acc, test_acc, y_val_true, y_val_pred, y_test_true, y_test_pred

In [ ]:
import dill
from keras import models #, layers
from keras.layers import LSTM, CuDNNLSTM, Dropout, Dense #, Concatenate 
from keras.models import load_model #, Model
from keras.optimizers import RMSprop #Adam #, RMSprop, Adadelta
from keras.regularizers import l2 #, l1
from keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler
from os import scandir
from os.path import exists, basename
import shutil 
import math
from numpy.random import seed
from tensorflow import set_random_seed 
import numpy as np
import tensorflow as tf
import random as rn
from keras import backend as K
import os

# earlyStop = EarlyStopping(monitor='val_acc', ## 4 COLAB 
earlyStop = EarlyStopping(monitor='val_accuracy', 
                          min_delta=0, 
                          patience=PATIENCE, 
                          verbose=1, 
                          mode='max', 
                          baseline=None, 
                          restore_best_weights=True)

progressCounter = 0

for LEARNING_RATE in LEARNING_RATE_VALUES:
    for MODEL_NAME in MODEL_NAME_VALUES:
        for MIRRORING in MIRRORING_VALUES:
            for STD_JITTERING in STD_JITTERING_VALUES:
                for LSTM_LAYERS in LSTM_LAYERS_VALUES:
                    for dropOut_idx, dropOut in enumerate(DROPOUT_VALUES):
                        for regularizer_idx, regularizer in enumerate(REGULARIZER_VALUES):
                            for HIDDEN_UNITS in HIDDEN_UNITS_VALUES:
                                for i, (folder_where_to_save, preprocess_functions, globalNorm) in enumerate(PREPROCESS_FUNCTION_TO_TEST):
                                    DROPOUT = dropOut
                                    RECURRENT_DROPOUT = dropOut
            #                         EPOCHS = round(REFERENCE_EPOCHS * (1+DROPOUT))
                                    progressCounter += 1
                                    folderPathWhereToSave = SAVED_MODEL_FOLDER+folder_where_to_save
                                    if folder_where_to_save.endswith("_lrScan/"):
                                        EPOCHS = SCAN_EPOCHS

                                    alreadyTrainedModel = [f for f in scandir(folderPathWhereToSave) if f.path.endswith(".h5")]

                                    ### SET SAVE FILE NAME ###
                                    if CONTINUE_TRAINING:
                                        saveFileName = CONTINUE_TRAINING
                                        saveFileName += "_THEN"
                                        saveFileName += "_MIRR" if MIRRORING else ""
                                        saveFileName += "_JIT_"+ str(STD_JITTERING) if STD_JITTERING>0 else "" 
                                        saveFileName += "_drop_"+ str(dropOut) if dropOut>0 else ""
                                    else:                                            
                                        saveFileName = getSaveFileName(modelName = MODEL_NAME,
                                                                       preprocess_functions = preprocess_functions,
                                                                       learning_rate= LEARNING_RATE,
                                                                       offset = LR_OFFSET,
                                                                       power = LR_POWER,
                                                                       numberOfLSTMLayers = LSTM_LAYERS,
                                                                       useCudaLSTM = USE_CuDNNLSTM,
                                                                       hiddenUnits = HIDDEN_UNITS,
                                                                       regularizerValue = regularizer,
                                                                       dropOut = DROPOUT, 
                                                                       recurrentDropOut = RECURRENT_DROPOUT,
                                                                       std_jittering = STD_JITTERING,
                                                                       mirroring = MIRRORING,
                                                                       normalizeGlobally = globalNorm)

                                    print("######## {}/{} - {} - {} ########".format(progressCounter, numberOfTests, saveFileName, folder_where_to_save))

                                    if saveFileName+".h5" in [m.name for m in alreadyTrainedModel]:
                                        print("### already done! ####")
                                        continue
                                    else:
                                        #to avoid that someone else will start the same fitting
                                        print("touching",folderPathWhereToSave+saveFileName+".h5")
                                        modelFileName = saveFileName+".h5"

                #                         pathToTouch = (folderPathWhereToSave+modelFileName).replace(" ","\ ") ## 4 COLAB 
                                        modelFileNamePath = folderPathWhereToSave+modelFileName
                                        !touch $modelFileNamePath
                                        print("## To be saved in [...]{} ###".format(folderPathWhereToSave[54:]))

                                    ### LOADING DATASET ###
                                    if folderPathWhereToSave.endswith("Senesi/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-SPLIT-SENESI-dataset.pickle"
                                    elif folderPathWhereToSave.endswith("top-models/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-SPLIT-dataset.pickle"      

                                    elif folderPathWhereToSave.endswith("Cross_subject_test/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT_TEST-dataset.pickle"            
                                    elif folderPathWhereToSave.endswith("Cross_view_test/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW_TEST-dataset.pickle"     

                                    elif any([folderPathWhereToSave.endswith(s) for s in ["Cross_subject/","Cross_subject_lrScan/"]]):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT-dataset.pickle"       
                                    elif any([folderPathWhereToSave.endswith(s) for s in ["Cross_view/","Cross_view_lrScan/"]]):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW-dataset.pickle"  

                                    elif folderPathWhereToSave.endswith("Cross_view_tough/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW_TOUGH-dataset.pickle"            
                                    elif folderPathWhereToSave.endswith("Cross_subject_tough/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT_TOUGH-dataset.pickle" 

                                    elif folderPathWhereToSave.endswith("Cross_view_mini/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW_MINI-dataset.pickle"            
                                    elif folderPathWhereToSave.endswith("Cross_subject_mini/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT_MINI-dataset.pickle"    

                                    elif folderPathWhereToSave.endswith("top+Senesi_Cross_view/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW_TOP+SENESI-dataset.pickle"            
                                    elif folderPathWhereToSave.endswith("top+Senesi_Cross_subject/"):
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT_TOP+SENESI-dataset.pickle" 

                                    elif folderPathWhereToSave.endswith("-SPLIT_DIFFERENT_FROM_DETECTRON-dataset)/") and MODEL_NAME == "PoseNet-101":
                                        datasetName = DATASET_FOLDER+MODEL_NAME+"-SPLIT_DIFFERENT_FROM_DETECTRON-dataset.pickle"
                                    else:
                                        raise Exception("result folder not correct")

                                    ### CHECK IF ALREADY PREPROCESSED DATASET EXISTS
                                    preprocessed_datasetName = basename(datasetName).replace("-dataset.pickle","")
                                    preprocessed_datasetName += "-MIRR" if MIRRORING else ""
                                    preprocessed_datasetName += "-STD_JIT_"+str(STD_JITTERING) if STD_JITTERING>0 else "" 
                                    for f in preprocess_functions:
                                        preprocessed_datasetName += "-"+f.__name__
                                    preprocessed_datasetName += "-dataset.pickle"

                                    preprocessed_datasetPath = DATASET_FOLDER+preprocessed_datasetName

#                                     print("###### NOT CHECKING IF THERE IS A PREPROCCED DATASET ######")
#                                     if False:
                                    if exists(preprocessed_datasetPath):
                                        print("#### Loading preprocessed dataset: ", preprocessed_datasetPath)
                                        with open(preprocessed_datasetPath,'rb') as file_in:
                                            prepDict = pickle.load(file_in)
                                            X_train = prepDict["X_train"]
                                            y_train = prepDict["y_train"]
                                            X_val = prepDict["X_val"]
                                            y_val = prepDict["y_val"]
                                            X_test = prepDict["X_test"]
                                            y_test = prepDict["y_test"]
                                            encodingLabels = prepDict["encodingLabels"]
                                    else:
                                        print("#### Loading dataset: ", datasetName)
                                        train_set, val_set, test_set = getData(datasetName)
                                        print("Preproccesing dataset...")
                    
                                        ##### DATA AUGMENTATION #####
                                        X_AXIS = 1 if MODEL_NAME == "PoseNet-101" else 0
                                        train_set = augmentData(train_set, xAxis = X_AXIS, mirroring = MIRRORING, std_jittering = STD_JITTERING)
                                        val_set = augmentData(val_set, xAxis = X_AXIS, mirroring = MIRRORING, std_jittering = STD_JITTERING)

                                        X_train, y_train, X_val, y_val, X_test, y_test, encodingLabels = preprocessData(train_set, 
                                                                                                                       val_set, 
                                                                                                                       test_set,  
                                                                                                                       preprocess_functions,
                                                                                                                       normaliseDatasetGlobally=globalNorm)


#                                     ###### CREATING GENERATORS #######
#                                     trainingGenerator = sampleGenerator(X_train, y_train, BATCH_SIZE, xAxis = 1 if MODEL_NAME == "PoseNet-101" else 0)
#                                     validationGenerator = sampleGenerator(X_val, y_val, BATCH_SIZE, xAxis = 1 if MODEL_NAME == "PoseNet-101" else 0)

#                                     assert False, "stop here"
    
                                    # MERGING TRAINNG AND VALIDATION
                                    if MERGING_TRAIN_VAL:
                                        print("## MERGING TRAIN+VAL ##")
                                        X_train = np.concatenate((X_train,X_val),axis=0)
                                        y_train = np.concatenate((y_train,y_val),axis=0)
                                        X_val,y_val = X_test,y_test
                                    
                                    # TO SOLVE THE DROPOUT LAYER SHAPE PROBLEM
                                    missing_train = (BATCH_SIZE-(X_train.shape[0]%BATCH_SIZE))%BATCH_SIZE
                                    X_train = np.concatenate((X_train,X_train[:missing_train]),axis=0)
                                    y_train = np.concatenate((y_train,y_train[:missing_train]),axis=0)

                                    ### DEFINING MODEL ###
                                    inputDim = (X_train.shape[1], X_train.shape[2])
                                    outputLen = len(y_train[0])

                                    ## callbacks and checkpoints
                                    minLossModelName = "{}-min_val_loss.hdf5".format(folderPathWhereToSave+saveFileName)
                                    checkpointLoss = ModelCheckpoint(minLossModelName, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

                                    maxAccModelName = "{}-max_val_acc.hdf5".format(folderPathWhereToSave+saveFileName)

                                    checkpointAcc = ModelCheckpoint(maxAccModelName, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max') 
                #                     checkpointAcc = ModelCheckpoint(maxAccModelName, monitor='val_acc', verbose=1, save_best_only=True, mode='max') ## 4 COLAB 


                                    callbacks_list = [checkpointLoss, checkpointAcc]

                                    if USE_SCHEDULER:
                                        callbacks_list.append(LearningRateScheduler(scheduler, verbose=1))
                                        callbacks_list.append(earlyStop)
                                    else:
                                        callbacks_list.append(earlyStop)


                                    model = models.Sequential()
                                    model.add(Dropout(DROPOUT, input_shape=inputDim, noise_shape=(BATCH_SIZE, 1, inputDim[1])))

                                    if USE_LSTM:
                                        if LSTM_LAYERS == 1:
                                            model.add(LSTM(HIDDEN_UNITS, recurrent_dropout = RECURRENT_DROPOUT))
                                        else:       
                                            model.add(LSTM(HIDDEN_UNITS, recurrent_dropout = RECURRENT_DROPOUT, return_sequences=True))
                                            for layerIdx in range(2, LSTM_LAYERS):
                                                model.add(LSTM(HIDDEN_UNITS, recurrent_dropout = RECURRENT_DROPOUT, return_sequences=True))
                                            model.add(LSTM(HIDDEN_UNITS, recurrent_dropout = RECURRENT_DROPOUT))    

                                    elif USE_CuDNNLSTM:
                                        print("#### REMOVED KERNEL REGULARIZER #######")
                                        reg=l2(regularizer) if regularizer > 0 else None

                                        if LSTM_LAYERS == 1:
                                            model.add(CuDNNLSTM(HIDDEN_UNITS, recurrent_regularizer = reg))
                                        else:
                                            model.add(CuDNNLSTM(HIDDEN_UNITS, recurrent_regularizer = reg, return_sequences=True))
                                            for layerIdx in range(2, LSTM_LAYERS):
                                                model.add(CuDNNLSTM(HIDDEN_UNITS, recurrent_regularizer = reg, return_sequences=True))                  
                                            model.add(CuDNNLSTM(HIDDEN_UNITS, recurrent_regularizer=reg))


                                    model.add(Dense(outputLen, activation='softmax'))

        #                             opt = Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999, amsgrad=False)
        #                             opt = Adadelta(learning_rate=LEARNING_RATE, rho=0.95)
                                    opt = RMSprop(lr=LEARNING_RATE, rho=0.9) ## 4 COLAB 
                #                     opt = RMSprop(lerning_rate=LEARNING_RATE, rho=0.9)

                                    model.compile(
                #                                 optimizer='rmsprop',
                                                optimizer=opt,
                                                loss='categorical_crossentropy',
                                                metrics=['accuracy'])
                    
                                    ## CONTINUE TRAINING
                                    if CONTINUE_TRAINING:
                                        print("Loading from",CONTINUE_TRAINING)
                                        continueTrainingModelName = "{}.h5".format(folderPathWhereToSave+CONTINUE_TRAINING)
                                        model.load_weights(continueTrainingModelName)
                                        previousModelName = continueTrainingModelName

                                    ## RECOVER FROM MIN LOSS MODEL IF EXISTS
                                    if exists(minLossModelName):
                                        print("#### Loading weights from MIN LOSS model")
                                        model.load_weights(minLossModelName)

                                    ## RECOVER FROM MIN LOSS MODEL IF EXISTS
                                    minLossBackupModelName = "{}-min_val_loss-BACKUP.hdf5".format(folderPathWhereToSave+saveFileName)
                                    if exists(maxAccModelName):
                                        print("#### Loading weights from MAX ACCURACY model")
                                        model.load_weights(minLossModelName)
                                        if exists(minLossModelName):
                                            print("#### BACKUP of weights for previous MIN LOSS model")
                                            shutil.copyfile(minLossModelName,minLossBackupModelName)

                                    ## Loading weights from previous model trained with an higher dropout rate
                                    loadedFromPreviousModel = False
                                    if (DROPOUT > 0 or RECURRENT_DROPOUT > 0) and LOAD_FROM_PREVIOUS_DROPOUT and USE_LSTM:
                                        for prev_dropOut in LOAD_FROM_PREVIOUS_DROPOUT:
                                            if prev_dropOut <= DROPOUT:
                                                continue
                                            previousModelName = getSaveFileName(modelName = MODEL_NAME, 
                                                                                preprocess_functions = preprocess_functions, 
                                                                                learning_rate= LEARNING_RATE,
                                                                                offset = LR_OFFSET,
                                                                                power = LR_POWER,
                                                                                numberOfLSTMLayers = LSTM_LAYERS, 
                                                                                useCudaLSTM = USE_CuDNNLSTM,
                                                                                hiddenUnits = HIDDEN_UNITS,
                                                                                regularizerValue = regularizer,
                                                                                dropOut = prev_dropOut, 
                                                                                recurrentDropOut = prev_dropOut,
                                                                                std_jittering = STD_JITTERING,
                                                                                mirroring = MIRRORING,
                                                                                normalizeGlobally = globalNorm)
                                            if exists(folderPathWhereToSave+previousModelName+".h5"):
                                                print("#### Loading weights from",previousModelName)
                                                model.load_weights(folderPathWhereToSave+previousModelName+".h5")
                                                loadedFromPreviousModel = True
                                                break


                                    # FIT ###
#                                     if MERGING_TRAIN_VAL:
#                                         model_history = model.fit(X_train, y_train,
#                                                         epochs=EPOCHS,
#                                                         batch_size=BATCH_SIZE,
#                                                         callbacks=callbacks_list,
#                                                         validation_data=(X_test, y_test)
#                                                        )
#                                     else:
                                    model_history = model.fit(X_train, y_train,
                                                        epochs=EPOCHS,
                                                        batch_size=BATCH_SIZE,
                                                        callbacks=callbacks_list,
                                                        validation_data=(X_val, y_val)
                                                       )    
                                    
#                                     print("### FITTING WITH GENERATORS  ####")
#                                     trainStepPerEpoch = int(len(X_train)/BATCH_SIZE)+1
#                                     valStepPerEpoch = int(len(X_val)/BATCH_SIZE)+1

#                                     history = model.fit_generator(trainingGenerator,
#                                                                   epochs=EPOCHS,
#                                                                   steps_per_epoch= trainStepPerEpoch,
#                                                                   callbacks=callbacks_list,
#                                                                   validation_data=validationGenerator,
#                                                                   validation_steps=valStepPerEpoch
#                                                                   )


                                    ### SAVE MODEL ###
                                    model.save(folderPathWhereToSave+saveFileName+".h5")

                                    ### SAVE HISTORY AND PREPROCESS FUNCTIONS ###
                                    env_functions = [one_hot_encoding, 
                                                      euclDistance,
                                                      paddingTrainValTest, 
                                                      getClosestNonZeroCoordinate,
                                                      removeZerosFromVideo,
                                                      getZeroStatsForDataset,
                                                      preprocessData,
                                                     ]

                                    historyToSave = {
                                        "acc" : model_history.history['accuracy'],
                                        "val_acc" : model_history.history['val_accuracy'],
                                        "loss" : model_history.history['loss'],
                                        "val_loss" : model_history.history['val_loss']
                                    }


                                    info_to_save = {"history": historyToSave,
                                                    "env_fun_DILL":[dill.dumps(x) for x in env_functions],
                                                    "spec_fun_DILL":[dill.dumps(x) for x in preprocess_functions]}
                                    
                                        
                                    if loadedFromPreviousModel or CONTINUE_TRAINING:
                                        info_to_save["loaded_from"] = previousModelName

                                    with open(folderPathWhereToSave+saveFileName+".pickle","wb") as handle:
                                        pickle.dump(info_to_save, handle) 
                                        print("## Saved in {} ###\n\n".format(folderPathWhereToSave+saveFileName+".pickle"))


                                    ### EVALUATING MODEL ### 
                                    model = load_model(maxAccModelName)
                                    !mv $maxAccModelName $modelFileNamePath
                                    val_acc, test_acc, y_val_true, y_val_pred, y_test_true, y_test_pred = getValTestAccuracy(model,X_val,y_val,X_test,y_test, encodingLabels)
                                    print("Model for MAX ACCURACY test_acc: {:.3f} val_acc: {:.3f}".format(test_acc*100,val_acc*100))

                                    if exists(minLossModelName):
                                        model_minLoss = load_model(minLossModelName)
                                        val_acc_minLoss, test_acc_minLoss, y_val_true, y_val_pred_minLoss, y_test_true, y_test_pred_minLoss = getValTestAccuracy(model_minLoss,X_val,y_val,X_test,y_test,encodingLabels)
                                        print("Model for MIN LOSS test_acc: {:.3f} val_acc: {:.3f}".format(test_acc_minLoss*100,val_acc_minLoss*100))
                                    else:
                                        print("Model for MIN LOSS NOT SAVED")
                                        val_acc_minLoss = math.inf

                                    if exists(minLossBackupModelName):
                                        model_minLoss_b = load_model(minLossBackupModelName)
                                        print("#### Evaluating BACKUP min loss model ####")
                                        val_acc_minLoss_b, test_acc_minLoss_b, y_val_true, y_val_pred_minLoss_b, y_test_true, y_test_pred_minLoss_b = getValTestAccuracy(model_minLoss_b,X_val,y_val,X_test,y_test,encodingLabels)
                                        print("Model for BACKUP MIN LOSS test_acc: {:.3f} val_acc: {:.3f}".format(test_acc_minLoss_b*100,val_acc_minLoss_b*100))
                                        if (val_acc_minLoss_b < val_acc_minLoss):
                                            print("### keeping BACKUP MIN LOSS model ####")
                                            val_acc_minLoss, test_acc_minLoss, y_val_pred_minLoss, y_test_pred_minLoss = val_acc_minLoss_b, test_acc_minLoss_b, y_val_pred_minLoss_b, y_test_pred_minLoss_b
                                            !mv $minLossBackupModelName $minLossModelName
                                        else:
                                            !rm $minLossBackupModelName

                                    ### CLEANING MAX ACC CHECKPOINT 
                                    ## 4 COLAB 
                #                     patToRemove = maxAccModelName.replace(" ","\ ")
            #                         patToRemove = maxAccModelName
            #                         !rm $patToRemove

                                    ### SAVING RESULTS ###
                                    if EPOCHS < 10:
                                        print("## SKIPPING SAVING. EPOCHS < 11")
                                        continue

                                    if exists(folderPathWhereToSave+"summaryResults.pickle"):
                                        print("Loading previous results...")
                                        with open(folderPathWhereToSave+"summaryResults.pickle","rb") as handle:
                                                results = pickle.load(handle)
                                    else:
                                        results = []

                                    results.append({"val_acc": val_acc, 
                                                    "test_acc": test_acc, 
                                                    "model_name": saveFileName, 
                                                    "y_val_true": y_val_true,
                                                    "y_val_pred": y_val_pred,
                                                    "y_test_true": y_test_true,
                                                    "y_test_pred": y_test_pred,

                                                    "val_acc_minLoss": val_acc_minLoss, 
                                                    "test_acc_minLoss": test_acc_minLoss, 
                                                    "y_val_pred_minLoss": y_val_pred_minLoss,
                                                    "y_test_pred_minLoss": y_test_pred_minLoss
                                                   })

                                    print("Dumping results...")
                                    with open(folderPathWhereToSave+"summaryResults.pickle","wb") as handle:
                                        pickle.dump(results, handle)

print("DONE!")


######## 1/72 - PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeToNextFrame-glob_norm-drop-0.15-reg-1e-05 - Cross_subject/ ########
touching /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_subject/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeToNextFrame-glob_norm-drop-0.15-reg-1e-05.h5
## To be saved in [...]/savedModels/Cross_subject/ ###
#### Loading dataset:  /data/students_home/amoscatelli/Desktop/actionAnalysis/datasets/PoseNet-101-CROSS_SUBJECT-dataset.pickle
train_set shape: (32256,)
train_set zero elements: 4070412/104805544 (3.9%)
val_set shape: (8064,)
val_set zero elements: 1067838/26143382 (4.1%)
test_set shape: (16560,)
test_set zero elements: 2234310/57232200 (3.9%)
Preproccesing dataset...
## OLD LENGHT OF DATSET: 32256
## NEW LENGHT OF DATSET: 32256
## OLD LENGHT OF DATSET: 8064
## NEW LENGHT OF DATSET: 8064
classes order:

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train on 40800 samples, validate on 16560 samples
Epoch 1/2000

Epoch 00001: LearningRateScheduler setting learning rate to 0.011.
40800/40800 [==============================] - 18s 449us/step - loss: 4.1005 - accuracy: 0.0148 - val_loss: 4.0962 - val_accuracy: 0.0167

Epoch 00001: val_loss improved from inf to 4.09616, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_subject/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeToNextFrame-glob_norm-drop-0.15-reg-1e-05-min_val_loss.hdf5

Epoch 00001: val_accuracy improved from -inf to 0.01667, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_subject/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeToNextFrame-glob_norm-drop-0.15-reg-1e-05-max_val_acc.hdf5
Epoch 2/2000

Epoch 00002: LearningRateScheduler setting learning rate to 0.0109201600000

40800/40800 [==============================] - 13s 322us/step - loss: 4.0947 - accuracy: 0.0156 - val_loss: 4.0936 - val_accuracy: 0.0168

Epoch 00016: val_loss improved from 4.09375 to 4.09365, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_subject/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeToNextFrame-glob_norm-drop-0.15-reg-1e-05-min_val_loss.hdf5

Epoch 00016: val_accuracy did not improve from 0.01691
Epoch 17/2000

Epoch 00017: LearningRateScheduler setting learning rate to 0.009760959999999999.
40800/40800 [==============================] - 12s 297us/step - loss: 4.0966 - accuracy: 0.0155 - val_loss: 4.1211 - val_accuracy: 0.0167

Epoch 00017: val_loss did not improve from 4.09365

Epoch 00017: val_accuracy did not improve from 0.01691
Epoch 18/2000

Epoch 00018: LearningRateScheduler setting learning rate to 0.009686239999999999.
40800/40800 [==============================

40800/40800 [==============================] - 12s 296us/step - loss: 2.1044 - accuracy: 0.3731 - val_loss: 2.0475 - val_accuracy: 0.3909

Epoch 00028: val_loss improved from 2.16460 to 2.04747, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_subject/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeToNextFrame-glob_norm-drop-0.15-reg-1e-05-min_val_loss.hdf5

Epoch 00028: val_accuracy improved from 0.35761 to 0.39094, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_subject/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeToNextFrame-glob_norm-drop-0.15-reg-1e-05-max_val_acc.hdf5
Epoch 29/2000

Epoch 00029: LearningRateScheduler setting learning rate to 0.008885440000000001.
40800/40800 [==============================] - 13s 318us/step - loss: 1.9572 - accuracy: 0.4129 - val_loss: 2.0068 -

...

38400/38400 [==============================] - 13s 350us/step - loss: 0.1984 - accuracy: 0.9430 - val_loss: 0.6925 - val_accuracy: 0.8237

Epoch 00413: val_loss did not improve from 0.62275

Epoch 00413: val_accuracy did not improve from 0.82737
Epoch 414/2000

Epoch 00414: LearningRateScheduler setting learning rate to 0.001.
38400/38400 [==============================] - 12s 324us/step - loss: 0.1959 - accuracy: 0.9456 - val_loss: 0.7078 - val_accuracy: 0.8224

Epoch 00414: val_loss did not improve from 0.62275

Epoch 00414: val_accuracy did not improve from 0.82737
Epoch 415/2000

Epoch 00415: LearningRateScheduler setting learning rate to 0.001.
38400/38400 [==============================] - 12s 325us/step - loss: 0.1985 - accuracy: 0.9441 - val_loss: 0.6844 - val_accuracy: 0.8261

Epoch 00415: val_loss did not improve from 0.62275

Epoch 00415: val_accuracy did not improve from 0.82737
Epoch 416/2000

Epoch 00416: LearningRateScheduler setting learning rate to 0.001.
38400/38400 [

classes order: ['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22'
 '23' '24' '25' '26' '27' '28' '29' '3' '30' '31' '32' '33' '34' '35' '36'
 '37' '38' '39' '4' '40' '41' '42' '43' '44' '45' '46' '47' '48' '49' '5'
 '50' '51' '52' '53' '54' '55' '56' '57' '58' '59' '6' '60' '7' '8' '9']
removing zeros from dataset
removing zeros from dataset
removing zeros from dataset
normalising EACH VIDEO, considering x and y INDIPENDENTLY
normalising EACH VIDEO, considering x and y INDIPENDENTLY
normalising EACH VIDEO, considering x and y INDIPENDENTLY
Adapting the data wrt to the 3 GLOBAL BARICENTERS of each VIDEO
Adapting the data wrt to the 3 GLOBAL BARICENTERS of each VIDEO
Adapting the data wrt to the 3 GLOBAL BARICENTERS of each VIDEO
normalising together train, val and test values BEFORE padding
train set shape: (30336, 300, 34, 2)
train set zero elements (after padding): 518362470 (83.76%)
val set shape: (7584, 300, 34, 2)
val set zero elements (after padding): 129687

38400/38400 [==============================] - 13s 330us/step - loss: 4.0956 - accuracy: 0.0155 - val_loss: 4.0945 - val_accuracy: 0.0167

Epoch 00014: val_loss improved from 4.09453 to 4.09450, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_view/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeTo3GlobalBaricentersOfVideo-glob_norm-drop-0.15-reg-1e-06-min_val_loss.hdf5

Epoch 00014: val_accuracy did not improve from 0.01667
Epoch 15/2000

Epoch 00015: LearningRateScheduler setting learning rate to 0.009911360000000001.
38400/38400 [==============================] - 13s 332us/step - loss: 4.0957 - accuracy: 0.0157 - val_loss: 4.0945 - val_accuracy: 0.0167

Epoch 00015: val_loss did not improve from 4.09450

Epoch 00015: val_accuracy did not improve from 0.01667
Epoch 16/2000

Epoch 00016: LearningRateScheduler setting learning rate to 0.009836000000000001.
38400/38400 [=================

38400/38400 [==============================] - 12s 323us/step - loss: 3.1609 - accuracy: 0.0817 - val_loss: 2.9489 - val_accuracy: 0.0902

Epoch 00031: val_loss improved from 3.11901 to 2.94890, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_view/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeTo3GlobalBaricentersOfVideo-glob_norm-drop-0.15-reg-1e-06-min_val_loss.hdf5

Epoch 00031: val_accuracy improved from 0.08808 to 0.09019, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_view/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeTo3GlobalBaricentersOfVideo-glob_norm-drop-0.15-reg-1e-06-max_val_acc.hdf5
Epoch 32/2000

Epoch 00032: LearningRateScheduler setting learning rate to 0.008673759999999999.
38400/38400 [==============================] - 13s 337us/step - loss: 3.0194 - accuracy: 0

38400/38400 [==============================] - 12s 323us/step - loss: 2.2279 - accuracy: 0.3003 - val_loss: 1.9875 - val_accuracy: 0.3737

Epoch 00040: val_loss improved from 2.22584 to 1.98751, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_view/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeTo3GlobalBaricentersOfVideo-glob_norm-drop-0.15-reg-1e-06-min_val_loss.hdf5

Epoch 00040: val_accuracy improved from 0.30200 to 0.37368, saving model to /data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/Cross_view/PoseNet-101-4L-CuDNNLSTM-HU_64-LR_0.01-OFF_0.001-POW_2-removeZerosFromDataset-normalizeVideosXYInpid-relativeTo3GlobalBaricentersOfVideo-glob_norm-drop-0.15-reg-1e-06-max_val_acc.hdf5
Epoch 41/2000

Epoch 00041: LearningRateScheduler setting learning rate to 0.008055999999999999.
38400/38400 [==============================] - 13s 337us/step - loss: 2.1590 - accuracy: 0

In [ ]:
 ### CHECK IF ALREADY PREPROCESSED DATASET EXISTS
preprocessed_datasetName = basename(datasetName).replace("-dataset.pickle","")
preprocessed_datasetName += "-MIRR" if mirroring else ""
preprocessed_datasetName += "-STD_JIT_"+str(std_jittering) if std_jittering>0 else "" 
for f in preprocess_functions:
    preprocessed_datasetName += "-"+f.__name__
    
preprocessed_datasetName += "-dataset.pickle"


preprocessed_datasetPath = DATASET_FOLDER+preprocessed_datasetName

## Continue Training

In [39]:
# # "det-3L-cuda-HU_64-LR_0,017-OFF_0,001-POW_1,5-rimoz_0-3BAR-norm-drop-0,15"
# MODEL_NAME = "keypoint_rcnn_X_101_32x8d_FPN_3x"
# modelToContinueTrain = MODEL_NAME+"-3L-CuDNNLSTM-HU_64-LR_0.017-OFF_0.001-POW_1.5-removeZerosFromDataset-relativeTo3BaricentersOfVideo-normalizeVideos-drop-0.15-reg-0"
# preprocess_functions = [removeZerosFromDataset,relativeTo3BaricentersOfVideo,normalizeVideos]

# folderToTrain = "Cross_subject/"
# CONFIGURATION_TO_CONTINUE_TRAIN = [(folderToTrain, modelToContinueTrain)]

# PATIENCE = 100
# EPOCHS = 50000
# BATCH_SIZE = 600
        
# LEARNING_RATE = 0.005
# USE_SCHEDULER = False
# ###### POLYNOMIAL SCHEDULER #############
# LR_OFFSET = 0.000
# LR_POWER = 1
# def polynomialScheduler(epoch, lr):
#     decay = (1 - (epoch / float(EPOCHS)))  ** LR_POWER
#     alpha = LEARNING_RATE * decay
#     return float(alpha) + LR_OFFSET
# scheduler = polynomialScheduler
# ##########################################




NameError: name 'relativeTo3BaricentersOfVideo' is not defined

In [ ]:
# import dill
# from keras import models #, layers
# from keras.layers import LSTM, CuDNNLSTM, Dropout, Dense #, Concatenate 
# from keras.models import load_model #, Model
# from keras.optimizers import RMSprop
# from keras.regularizers import l2 #, l1
# from keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler
# from os import scandir
# from os.path import exists, basename
# import shutil 
# import math
# from numpy.random import seed
# from tensorflow import set_random_seed 
# import numpy as np
# import tensorflow as tf
# import random as rn
# from keras import backend as K
# import os


# # earlyStop = EarlyStopping(monitor='val_acc', ## 4 COLAB 
# earlyStop = EarlyStopping(monitor='val_accuracy', 
#                           min_delta=0, 
#                           patience=PATIENCE, 
#                           verbose=1, 
#                           mode='max', 
#                           baseline=None, 
#                           restore_best_weights=True)

# for idx_configuration, (folderToTrain,modelToTrain) in enumerate(CONFIGURATION_TO_CONTINUE_TRAIN):
#     print("######## {}/{} - {} - {} ########".format(idx_configuration, len(CONFIGURATION_TO_CONTINUE_TRAIN),cleanForExcel(modelToTrain),folderToTrain))

#     folderPathWhereToSave = SAVED_MODEL_FOLDER+folderToTrain
#     if not exists(folderPathWhereToSave+modelToTrain+".h5"):
#         print("model {} not existend in folder {}".format(modelToTrain,folderToTrain))
#         continue
          
#     alreadyTrainedModel = [f for f in scandir(folderPathWhereToSave) if f.path.endswith(".h5")]
    
#     saveFileName = modelToTrain+"-THEN_LR_"+str(LEARNING_RATE)
          
#     if saveFileName+".h5" in [m.name for m in alreadyTrainedModel]:
#         print("### already done! ####")
#         continue
#     else:
#         #to avoid that someone else will start the same fitting
#         print("touching",folderPathWhereToSave+saveFileName+".h5")
#         modelFileName = saveFileName+".h5"

# #       pathToTouch = (folderPathWhereToSave+modelFileName).replace(" ","\ ") ## 4 COLAB 
#         modelFileNamePath = folderPathWhereToSave+modelFileName
#         !touch $modelFileNamePath
#         print("## To be saved in [...]{} ###".format(folderPathWhereToSave[54:]))

#     ### LOADING DATASET ###
#     if folderPathWhereToSave.endswith("Senesi/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-SPLIT-SENESI-dataset.pickle"
#     elif folderPathWhereToSave.endswith("top-models/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-SPLIT-dataset.pickle"      

#     elif folderPathWhereToSave.endswith("Cross_subject_test/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT_TEST-dataset.pickle"            
#     elif folderPathWhereToSave.endswith("Cross_view_test/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW_TEST-dataset.pickle"     

#     elif any([folderPathWhereToSave.endswith(s) for s in ["Cross_subject/","Cross_subject_lrScan/"]]):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT-dataset.pickle"       
#     elif any([folderPathWhereToSave.endswith(s) for s in ["Cross_view/","Cross_view_lrScan/"]]):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW-dataset.pickle"  

#     elif folderPathWhereToSave.endswith("Cross_view_tough/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW_TOUGH-dataset.pickle"            
#     elif folderPathWhereToSave.endswith("Cross_subject_tough/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT_TOUGH-dataset.pickle" 

#     elif folderPathWhereToSave.endswith("Cross_view_mini/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW_MINI-dataset.pickle"            
#     elif folderPathWhereToSave.endswith("Cross_subject_mini/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT_MINI-dataset.pickle"    

#     elif folderPathWhereToSave.endswith("top+Senesi_Cross_view/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_VIEW_TOP+SENESI-dataset.pickle"            
#     elif folderPathWhereToSave.endswith("top+Senesi_Cross_subject/"):
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-CROSS_SUBJECT_TOP+SENESI-dataset.pickle" 

#     elif folderPathWhereToSave.endswith("-SPLIT_DIFFERENT_FROM_DETECTRON-dataset)/") and MODEL_NAME == "PoseNet-101":
#         datasetName = DATASET_FOLDER+MODEL_NAME+"-SPLIT_DIFFERENT_FROM_DETECTRON-dataset.pickle"
#     else:
#         raise Exception("result folder not correct")

#     ### CHECK IF ALREADY PREPROCESSED DATASET EXISTS
#     preprocessed_datasetName = basename(datasetName).replace("-dataset.pickle","")
#     for f in preprocess_functions:
#         preprocessed_datasetName += "-"+f.__name__
#     preprocessed_datasetName += "-dataset.pickle"

#     preprocessed_datasetPath = DATASET_FOLDER+preprocessed_datasetName

#     if exists(preprocessed_datasetPath):
#         print("#### Loading preprocessed dataset: ", preprocessed_datasetPath)
#         with open(preprocessed_datasetPath,'rb') as file_in:
#             prepDict = pickle.load(file_in)
#             X_train = prepDict["X_train"]
#             y_train = prepDict["y_train"]
#             X_val = prepDict["X_val"]
#             y_val = prepDict["y_val"]
#             X_test = prepDict["X_test"]
#             y_test = prepDict["y_test"]
#             encodingLabels = prepDict["encodingLabels"]
#     else:
#         print("#### Loading dataset: ", datasetName)
#         train_set, val_set, test_set = getData(datasetName)
#         print("Preproccesing dataset...")
#         X_train, y_train, X_val, y_val, X_test, y_test, encodingLabels = preprocessData(train_set, 
#                                                                                        val_set, 
#                                                                                        test_set,  
#                                                                                        preprocess_functions)

#     # TO SOLVE THE DROPOUT LAYER SHAPE PROBLEM
#     missing_train = (BATCH_SIZE-(X_train.shape[0]%BATCH_SIZE))%BATCH_SIZE
#     X_train = np.concatenate((X_train,X_train[:missing_train]),axis=0)
#     y_train = np.concatenate((y_train,y_train[:missing_train]),axis=0)

#     ### DEFINING MODEL ###
#     inputDim = (X_train.shape[1], X_train.shape[2])
#     outputLen = len(y_train[0])

#     ## callbacks and checkpoints
#     minLossModelName = "{}-min_val_loss.hdf5".format(folderPathWhereToSave+saveFileName)
#     checkpointLoss = ModelCheckpoint(minLossModelName, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

#     maxAccModelName = "{}-max_val_acc.hdf5".format(folderPathWhereToSave+saveFileName)

#     checkpointAcc = ModelCheckpoint(maxAccModelName, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max') 
# #   checkpointAcc = ModelCheckpoint(maxAccModelName, monitor='val_acc', verbose=1, save_best_only=True, mode='max') ## 4 COLAB 


#     callbacks_list = [checkpointLoss, checkpointAcc]

#     if USE_SCHEDULER:
#         callbacks_list.append(LearningRateScheduler(scheduler, verbose=1))
#     else:
#         callbacks_list.append(earlyStop)

            
#     model = load_model(folderPathWhereToSave+modelToTrain+".h5")

#     rmpsprop = RMSprop(lr=LEARNING_RATE, rho=0.9) 

#     model.compile(
# #               optimizer='rmsprop',
#                 optimizer=rmpsprop,
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])


#     ## RECOVER FROM MIN LOSS MODEL IF EXISTS
#     if exists(minLossModelName):
#         print("#### Loading weights from MIN LOSS model")
#         model.load_weights(minLossModelName)

#     ## RECOVER FROM MIN LOSS MODEL IF EXISTS
#     minLossBackupModelName = "{}-min_val_loss-BACKUP.hdf5".format(folderPathWhereToSave+saveFileName)
#     if exists(maxAccModelName):
#         print("#### Loading weights from MAX ACCURACY model")
#         model.load_weights(minLossModelName)
#         if exists(minLossModelName):
#             print("#### BACKUP of weights for previous MIN LOSS model")
#             shutil.copyfile(minLossModelName,minLossBackupModelName)


#     ## FIT ###    
#     model_history = model.fit(X_train, y_train,
#                         epochs=EPOCHS,
#                         batch_size=BATCH_SIZE,
#                         callbacks=callbacks_list,
#                         validation_data=(X_val, y_val)
#                        )

#     ### SAVE MODEL ###
#     model.save(folderPathWhereToSave+saveFileName+".h5")

#     ### SAVE HISTORY AND PREPROCESS FUNCTIONS ###
#     env_functions = [one_hot_encoding, 
#                       euclDistance,
#                       paddingTrainValTest, 
#                       getClosestNonZeroCoordinate,
#                       removeZerosFromVideo,
#                       getZeroStatsForDataset,
#                       preprocessData,
#                      ]

#     historyToSave = {
#         "acc" : model_history.history['accuracy'],
#         "val_acc" : model_history.history['val_accuracy'],
#         "loss" : model_history.history['loss'],
#         "val_loss" : model_history.history['val_loss']
#     }


#     info_to_save = {"history": historyToSave,
#                     "env_fun_DILL":[dill.dumps(x) for x in env_functions],
#                     "spec_fun_DILL":[dill.dumps(x) for x in preprocess_functions]}

#     info_to_save["loaded_from"] = folderPathWhereToSave+modelToTrain

#     with open(folderPathWhereToSave+saveFileName+".pickle","wb") as handle:
#         pickle.dump(info_to_save, handle) 
#         print("## Saved in {} ###\n\n".format(folderPathWhereToSave+saveFileName+".pickle"))


#     ### EVALUATING MODEL ### 
#     model = load_model(maxAccModelName)
#     !mv $maxAccModelName $modelFileNamePath
#     val_acc, test_acc, y_val_true, y_val_pred, y_test_true, y_test_pred = getValTestAccuracy(model,X_val,y_val,X_test,y_test, encodingLabels)
#     print("Model for MAX ACCURACY test_acc: {:.3f} val_acc: {:.3f}".format(test_acc*100,val_acc*100))

#     if exists(minLossModelName):
#         model_minLoss = load_model(minLossModelName)
#         val_acc_minLoss, test_acc_minLoss, y_val_true, y_val_pred_minLoss, y_test_true, y_test_pred_minLoss = getValTestAccuracy(model_minLoss,X_val,y_val,X_test,y_test,encodingLabels)
#         print("Model for MIN LOSS test_acc: {:.3f} val_acc: {:.3f}".format(test_acc_minLoss*100,val_acc_minLoss*100))
#     else:
#         print("Model for MIN LOSS NOT SAVED")
#         val_acc_minLoss = math.inf

#     if exists(minLossBackupModelName):
#         model_minLoss_b = load_model(minLossBackupModelName)
#         print("#### Evaluating BACKUP min loss model ####")
#         val_acc_minLoss_b, test_acc_minLoss_b, y_val_true, y_val_pred_minLoss_b, y_test_true, y_test_pred_minLoss_b = getValTestAccuracy(model_minLoss_b,X_val,y_val,X_test,y_test,encodingLabels)
#         print("Model for BACKUP MIN LOSS test_acc: {:.3f} val_acc: {:.3f}".format(test_acc_minLoss_b*100,val_acc_minLoss_b*100))
#         if (val_acc_minLoss_b < val_acc_minLoss):
#             print("### keeping BACKUP MIN LOSS model ####")
#             val_acc_minLoss, test_acc_minLoss, y_val_pred_minLoss, y_test_pred_minLoss = val_acc_minLoss_b, test_acc_minLoss_b, y_val_pred_minLoss_b, y_test_pred_minLoss_b
#             !mv $minLossBackupModelName $minLossModelName
#         else:
#             !rm $minLossBackupModelName


#     ### SAVING RESULTS ###
#     if EPOCHS < 10:
#         print("## SKIPPING SAVING. EPOCHS < 11")
#         continue

#     if exists(folderPathWhereToSave+"summaryResults.pickle"):
#         print("Loading previous results...")
#         with open(folderPathWhereToSave+"summaryResults.pickle","rb") as handle:
#                 results = pickle.load(handle)
#     else:
#         results = []

#     results.append({"val_acc": val_acc, 
#                     "test_acc": test_acc, 
#                     "model_name": saveFileName, 
#                     "y_val_true": y_val_true,
#                     "y_val_pred": y_val_pred,
#                     "y_test_true": y_test_true,
#                     "y_test_pred": y_test_pred,

#                     "val_acc_minLoss": val_acc_minLoss, 
#                     "test_acc_minLoss": test_acc_minLoss, 
#                     "y_val_pred_minLoss": y_val_pred_minLoss,
#                     "y_test_pred_minLoss": y_test_pred_minLoss
#                    })

#     print("Dumping results...")
#     with open(folderPathWhereToSave+"summaryResults.pickle","wb") as handle:
#         pickle.dump(results, handle)

# print("DONE!")


## Save preprocced Dataset

In [43]:
from os.path import exists, basename
newDatasetName = basename(datasetName).replace("-dataset.pickle","")
newDatasetName += "-MIRR" if MIRRORING else ""
newDatasetName += "-STD_JIT_"+str(STD_JITTERING) if STD_JITTERING>0 else ""
for f in preprocess_functions:
    newDatasetName += "-"+f.__name__
newDatasetName += "-dataset.pickle"

newDatasetPath = DATASET_FOLDER+newDatasetName
newDatasetPath
print("Dumping results on ",newDatasetPath)
with open(newDatasetPath,"wb") as handle:
    pickle.dump({"X_train": X_train,
                 "y_train": y_train,
                 "X_val": X_val,
                 "y_val": y_val, 
                 "X_test": X_test,
                 "y_test": y_test, 
                 "encodingLabels": encodingLabels}, handle)
print("Done")

Dumping results on  /data/students_home/amoscatelli/Desktop/actionAnalysis/datasets/keypoint_rcnn_X_101_32x8d_FPN_3x-CROSS_SUBJECT-MIRR-STD_JIT_2-removeZerosFromDataset-relativeTo3GlobalBaricentersOfVideo-normalizeVideos-dataset.pickle


OverflowError: cannot serialize a bytes object larger than 4 GiB

## Shutdown session

In [ ]:
%%javascript 
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>

In [27]:
import dill
from keras import models #, layers
from keras.layers import LSTM, CuDNNLSTM, Dropout, Dense #, Concatenate 
from keras.models import load_model #, Model
from keras.optimizers import RMSprop
from keras.regularizers import l2 #, l1
from keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler
# import random
from os import scandir
from os.path import exists
import shutil 
from numpy.random import seed
from tensorflow import set_random_seed 

# setting SEED in order to initialize the networks always in the same way
seed(2)
set_random_seed(2)

USE_LSTM = False
USE_CuDNNLSTM = not USE_LSTM
LSTM_LAYERS = 1
HIDDEN_UNITS = 64
### inputDim (251, 68)
# 122 4L -> 335k
# 155 3L -> 334k
# 256 2L -> 336k

### inputDim (251, 17*16)
# 122 4L -> 434k
# 155 3L -> 461k
# 256 2L -> 545k

DROPOUT = 0
RECURRENT_DROPOUT = 0
regularizer = 0
LEARNING_RATE = 0.001
BATCH_SIZE = 80
inputDim = (251, 68) # Mini
outputLen = 8

# inputDim = (300, 68) # 60 azioni
# outputLen = 60


model = models.Sequential()
model.add(Dropout(DROPOUT, input_shape=inputDim, noise_shape=(BATCH_SIZE, 1, inputDim[1])))

if USE_LSTM:
    if LSTM_LAYERS == 1:
        model.add(LSTM(HIDDEN_UNITS, recurrent_dropout = RECURRENT_DROPOUT))
    else:       
        model.add(LSTM(HIDDEN_UNITS, recurrent_dropout = RECURRENT_DROPOUT, return_sequences=True))
        for layerIdx in range(2, LSTM_LAYERS):
            model.add(LSTM(HIDDEN_UNITS, recurrent_dropout = RECURRENT_DROPOUT, return_sequences=True))
        model.add(LSTM(HIDDEN_UNITS, recurrent_dropout = RECURRENT_DROPOUT))    

elif USE_CuDNNLSTM:
    reg=l2(regularizer) if regularizer > 0 else None

    if LSTM_LAYERS == 1:
        model.add(CuDNNLSTM(HIDDEN_UNITS, kernel_regularizer = reg, recurrent_regularizer = reg))
    else:
        model.add(CuDNNLSTM(HIDDEN_UNITS, kernel_regularizer = reg, recurrent_regularizer = reg, return_sequences=True))
        for layerIdx in range(2, LSTM_LAYERS):
            model.add(CuDNNLSTM(HIDDEN_UNITS, kernel_regularizer = reg, recurrent_regularizer = reg, return_sequences=True))                  
        model.add(CuDNNLSTM(HIDDEN_UNITS, kernel_regularizer = reg, recurrent_regularizer=reg))


model.add(Dense(outputLen, activation='softmax'))

rmpsprop = RMSprop(learning_rate=LEARNING_RATE, rho=0.9)

model.compile(
#                                 optimizer='rmsprop',
            optimizer=rmpsprop,
            loss='categorical_crossentropy',
            metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_6 (Dropout)          (None, 251, 68)           0         
_________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)     (None, 64)                34304     
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 520       
Total params: 34,824
Trainable params: 34,824
Non-trainable params: 0
_________________________________________________________________


In [28]:
# weight1 = model.get_weights()
weight2 = model.get_weights()
# for layer in model.layers:
#     weights = layer.get_weights() # list of numpy arrays

## Removing accessories file for low results

In [ ]:
# from os.path import isfile, isdir, join, exists,getsize,basename
# from os import scandir
# import pickle

# NUMBER_OF_BEST_MODEL_TO_SAVE = 20

# SAVED_MODEL_FOLDER = "/data/students_home/amoscatelli/Desktop/actionAnalysis/savedModels/"
# # SAVED_MODEL_FOLDER += "Senesi/"
# # SAVED_MODEL_FOLDER += "top-models/"
# SAVED_MODEL_FOLDER += "Cross_view_mini/"
# # SAVED_MODEL_FOLDER += "Cross_subject_mini/"


# accessoryFileList = [f.path for f in scandir(SAVED_MODEL_FOLDER) 
#                        if f.path.endswith(".pickle") 
#                        and "summaryResult" not in basename(f)]

# with open(SAVED_MODEL_FOLDER+"summaryResults.pickle","rb") as handle:
#         loadedResults = pickle.load(handle)
        
# loadedResults.sort(key=lambda x : x["test_acc"], reverse=True)
# assert NUMBER_OF_BEST_MODEL_TO_SAVE > 10
# bestModels = [r['model_name'] for r in loadedResults[:NUMBER_OF_BEST_MODEL_TO_SAVE]]

# for accessoryFile in accessoryFileList:
#     modelNameToCheck = basename(accessoryFile).replace(".pickle","")
#     if modelNameToCheck not in bestModels:
#         print("Removing",accessoryFile)
#         !rm $accessoryFile